In [13]:
import pandas as pd
import numpy as np
import re

In [14]:
df=pd.read_csv('new_data.csv')

/var/folders/9g/mm9pdxg52dn6rg0rtfk14syc0000gn/T/ipykernel_36263/889337693.py:1: DtypeWarning: Columns (102,116,122,123,124,125,126,128,129) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('new_data.csv')


In [15]:
df.columns

Index(['Unnamed: 0', 'Available as of', 'Property name', 'Construction year',
       'Floor', 'Number of floors', 'Building condition',
       'Number of frontages', 'Covered parking spaces',
       'Outdoor parking spaces',
       ...
       'Percentage rented', 'Age of annuitant', 'EPC description',
       'Venue of the sale', 'Single session', 'Terms of visit',
       'Starting price', '0', 'Maximum duration of annuity',
       'Date of the sale'],
      dtype='object', length=130)

In [16]:
df.shape

(19291, 130)

In [17]:
filtered_columns=["Unnamed: 0", "Zip-code", "City", "Type of Property", 
                  "Subtype of Property", "Price", "Construction year", "Bedrooms","Living area",
                  "Kitchen type", "Furnished", "How many fireplaces?", "Terrace","Terrace surface",
                  "Garden","Garden surface","Surface of the plot","Number of frontages",
                  "Swimming pool", "Building condition", "Elevator", "Basement", "Cadastral income",
                  "Primary energy consumption","CO₂ emission", "Energy class", "Heating type"]
    
df = df[filtered_columns]

In [18]:
df=df.drop_duplicates()

In [19]:
df=df.rename(columns = {'Unnamed: 0':'Property ID', 'Zip-code':'Zip code'})

In [20]:
df=df.rename(columns = {'Price':'Price (€)', 'Living area':'Living area (m²)', 'Terrace surface':'Terrace surface (m²)', 
                        'Garden surface':'Garden surface (m²)', 'Surface of the plot': 'Surface of the plot (m²)', 'Cadastral income':'Cadastral income (€)',
                        'Primary energy consumption':'Primary energy consumption (kWh/m²)', 'CO₂ emission':'CO₂ emission (kg CO₂/m²)',
                          })

In [21]:
df=df.drop_duplicates()

In [22]:
df['How many fireplaces?']=df['How many fireplaces?'].replace(np.nan,0,regex=True)
df['How many fireplaces?']=df['How many fireplaces?'].replace('Yes',1,regex=True)
df['How many fireplaces?']=df['How many fireplaces?'].replace('No',0,regex=True)
df['Swimming pool']=df['Swimming pool'].replace(np.nan,0,regex=True)
df['Swimming pool']=df['Swimming pool'].replace('Yes',1,regex=True)
df['Swimming pool']=df['Swimming pool'].replace('No',0,regex=True)
df['Elevator']=df['Elevator'].replace(np.nan,0,regex=True)
df['Elevator']=df['Elevator'].replace('Yes',1,regex=True)
df['Elevator']=df['Elevator'].replace('No',0,regex=True)
df['Basement']=df['Basement'].replace(np.nan,0,regex=True)
df['Basement']=df['Basement'].replace('Yes',1,regex=True)
df['Basement']=df['Basement'].replace('No',0,regex=True)
df['Furnished']=df['Furnished'].replace(np.nan,0,regex=True)
df['Furnished']=df['Furnished'].replace('Yes',1,regex=True)
df['Furnished']=df['Furnished'].replace('No',0,regex=True)


In [25]:
df=df.dropna(subset=['Price (€)', 'Bedrooms', 'Living area (m²)', 'Subtype of Property'])


In [29]:
df['Price (€)'] = df['Price (€)'].str.split(' ').str[-2]
df['Price (€)'] = pd.to_numeric(df['Price (€)'])
df['Living area (m²)'] = df['Living area (m²)'].str.split(' ').str[0]
df['Garden surface (m²)'] = df['Garden surface (m²)'].str.split(' ').str[0]
df['Terrace surface (m²)'] = df['Terrace surface (m²)'].str.split(' ').str[0]
df['Surface of the plot (m²)'] = df['Surface of the plot (m²)'].str.split(' ').str[0]
df['Cadastral income (€)'] = df['Cadastral income (€)'].str.split(' ').str[-2]
df['Primary energy consumption (kWh/m²)'] = df['Primary energy consumption (kWh/m²)'].str.split(' ').str[0]
df['CO₂ emission (kg CO₂/m²)'] = df['CO₂ emission (kg CO₂/m²)'].str.split(' ').str[0]

In [30]:
filter_G = df["Garden surface (m²)"].isnull()
df.loc[~filter_G,'Garden'] = 'Yes'
df.loc[df['Garden'] == 'Yes', 'Garden'] = 1
df.loc[df["Garden"].isnull(), 'Garden'] = 0
df.loc[df["Garden surface (m²)"].isnull(), 'Garden surface (m²)'] = 0

In [31]:
filter_G = df["Terrace surface (m²)"].isnull()
df.loc[~filter_G,'Terrace'] = 'Yes'
df.loc[df['Terrace'] == 'Yes', 'Terrace'] = 1
df.loc[df["Terrace"].isnull(), 'Terrace'] = 0
df.loc[df["Terrace surface (m²)"].isnull(), 'Terrace surface (m²)'] = 0

In [32]:
df['Primary energy consumption (kWh/m²)']=df['Primary energy consumption (kWh/m²)'].replace('Not',np.nan)
df['CO₂ emission (kg CO₂/m²)']=df['CO₂ emission (kg CO₂/m²)'].replace('Not',np.nan)
df['Energy class']=df['Energy class'].replace('Not',np.nan)

In [33]:
df = df.astype({"Living area (m²)":"float", "Terrace":"float", "Garden":"float",
                "Terrace surface (m²)":"float",
                "Garden surface (m²)":"float","Surface of the plot (m²)":"float",
                "Primary energy consumption (kWh/m²)":"float", "Cadastral income (€)":"float",
                "CO₂ emission (kg CO₂/m²)":"float"})

In [34]:
df['Energy class']=df['Energy class'].replace('Not specified',np.nan)

In [35]:
kitchen_type_scale={'USA hyper equipped':3, 'USA installed':2, 'USA semi equipped':1, 'USA uninstalled':0,
 'Hyper equipped':3, 'Installed':2, 'Semi equipped':1, 'Not installed':0}

df['Kitchen type scale'] = df['Kitchen type'].map(kitchen_type_scale)

In [36]:
building_condition_scale={'As new':6,'Just renovated':5, 'Good':4, 'To renovate':2,
 'To restore':1, 'To be done up':3}

df['Building condition scale'] = df['Building condition'].map(building_condition_scale)

In [37]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [38]:
df.to_csv("updated_cleaned_data.csv", index=False)

In [43]:
df.shape

(15341, 29)